# Getting Started

For a quick start, we compare the different algorithms for deconvolution on the IRIS data set, estimating the probability density of Iris plant types.

In [1]:
# load the example data
using MLDataUtils
X, y_labels, _ = load_iris()

# discretize the target quantity (for numerical values, we'd use LinearDiscretizer)
using Discretizers: encode, CategoricalDiscretizer
y = encode(CategoricalDiscretizer(y_labels), y_labels) # vector of target value indices

# have a look at the content of y
unique(y) # its just indices

3-element Array{Int64,1}:
 1
 2
 3

In [2]:
# Split the data into training and observed data sets.
# 
# The matrices MLDataUtils expects are transposed, by default.
# Thus, we have to be explicit about obsdim = 1. Note that
# CherenkovDeconvolution.jl follows the convention of ScikitLearn.jl
# (and others), which is size(X_train) == (n_examples, n_features).
# 
# MLDataUtils unfortunately assumes size(X_train) == (n_features, n_examples),
# but obsdim = 1 fixes this assumption.
# 
using Random; Random.seed!(42) # make split reproducible
(X_train, y_train), (X_data, y_data) = splitobs(shuffleobs((X', y), obsdim = 1), obsdim = 1);

## Deconvolution with DSEA

The Dortmund Spectrum Estimation Algorithm (DSEA) reconstructs the target density from classifier predictions on the target quantity of individual examples. CherenkovDeconvolution.jl implements the improved version DSEA+, which is extended by adaptive step sizes and a fixed reweighting of examples.

In [3]:
using ScikitLearn, CherenkovDeconvolution
@sk_import naive_bayes : GaussianNB

# deconvolve with a Naive Bayes classifier
dsea = DSEA(GaussianNB()) # instantiate the deconvolution method
f_dsea = deconvolve(dsea, X_data, X_train, y_train) # returns a vector of target value probabilities

┌ Info: DSEA iteration 1/1 uses alpha = 1.0 (chi2s = 0.0028011676660929232)
└ @ CherenkovDeconvolution.Methods /home/bunse/.julia/dev/CherenkovDeconvolution/src/methods/dsea.jl:169


3-element Array{Float64,1}:
 0.3333333327844613 
 0.3549289670574494 
 0.31173770015808927

In [4]:
# compare the result to the true target distribution, which we are estimating
f_true = DeconvUtil.fit_pdf(y_data) # f_dsea is almost equal to f_true!

3-element Array{Float64,1}:
 0.3333333333333333 
 0.35555555555555557
 0.3111111111111111 

##  Classical Deconvolution-Algorithms

The Regularized Unfolding (RUN) fits the density distribution `f` to the convolution model `g = R * f`, using maximum likelihood. The regularization strength is configured with `n_df`, the effective number of degrees of freedom in the second-order local model of the solution.

The Iterative Bayesian Unfolding (IBU) reconstructs the target density by iteratively applying Bayes' rule to the conditional probabilities contained in the detector response matrix.

The SVD-based method computes the singular value decomposition of the detector response matrix `R`, fitting `f` according to the method of least squares.

In [5]:
#
# The classical algorithms are only applicable with a single discrete observable dimension.
# In order to obtain a dimension that contains as much information as possible, we discretize
# the feature space with a decision tree, using its leaves as clusters. The cluster indices
# are the discrete values of the observed dimension.
#
binning = TreeBinning(6) # obtain (up to) 6 clusters

# inspect the way in which the TreeBinning discretizes the data
td = BinningDiscretizer(binning, X_train, y_train) # fit the tree with labeled data
x_train = encode(td, X_train) # apply it to the feature vectors
unique(x_train) # the result are the cluster indices

6-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6

In [6]:
# RUN and IBU need a binning instead of a classifier
f_ibu = deconvolve(IBU(binning), X_data, X_train, y_train)

3-element Array{Float64,1}:
 0.3333333333333333
 0.3463872738499534
 0.3202793928167133

In [7]:
f_run = deconvolve(RUN(binning), X_data, X_train, y_train)

┌ Warning: Limiting RUN to 3 of 6 observeable non-zero bins
└ @ CherenkovDeconvolution.Methods /home/bunse/.julia/dev/CherenkovDeconvolution/src/methods/run.jl:110
┌ Warning: Performing acceptance correction regularisation requires a given acceptance_correction object
└ @ CherenkovDeconvolution.Methods /home/bunse/.julia/dev/CherenkovDeconvolution/src/methods/run.jl:137


3-element Array{Float64,1}:
 0.32062181199902845
 0.34272528540199176
 0.33665290259897984

In [8]:
f_p_run = deconvolve(PRUN(binning), X_data, X_train, y_train)

┌ Warning: Limiting PRUN to 3 of 6 observeable non-zero bins
└ @ CherenkovDeconvolution.Methods /home/bunse/.julia/dev/CherenkovDeconvolution/src/methods/prun.jl:113
┌ Warning: Performing acceptance correction regularisation requires a given acceptance_correction object
└ @ CherenkovDeconvolution.Methods /home/bunse/.julia/dev/CherenkovDeconvolution/src/methods/prun.jl:149


3-element Array{Float64,1}:
 0.3206218725187752 
 0.34272523890463785
 0.3366528885765869 

In [9]:
f_svd = deconvolve(SVD(binning), X_data, X_train, y_train)

3-element Array{Float64,1}:
 0.3206218119990284 
 0.3427252854019929 
 0.33665290259897873

## More Information

In [10]:
?DSEA # You can find more information in the documentation

search: DSEA dsea f_dsea GridSearch DenseArray DenseMatrix DenseVecOrMat



```
DSEA(classifier; kwargs...)
```

The *DSEA/DSEA+* deconvolution method, embedding the given `classifier`.

**Keyword arguments**

  * `f_0 = ones(m) ./ m` defines the prior, which is uniform by default
  * `fixweighting = true` sets, whether or not the weight update fix is applied. This fix is proposed in my Master's thesis and in the corresponding paper.
  * `stepsize = DEFAULT_STEPSIZE` is the step size taken in every iteration.
  * `smoothing = Base.identity` is a function that optionally applies smoothing in between iterations.
  * `K = 1` is the maximum number of iterations.
  * `epsilon = 0.0` is the minimum symmetric Chi Square distance between iterations. If the actual distance is below this threshold, convergence is assumed and the algorithm stops.
  * `inspect = nothing` is a function `(f_k::Vector, k::Int, chi2s::Float64, alpha_k::Float64) -> Any` optionally called in every iteration.
  * `return_contributions = false` sets, whether or not the contributions of individual examples in `X_data` are returned as a tuple together with the deconvolution result.


In [11]:
?IBU

search: IBU f_ibu IOBuffer @inbounds PipeBuffer



```
IBU(binning; kwargs...)
```

The *Iterative Bayesian Unfolding* deconvolution method, using a `binning` to discretize the observable features.

**Keyword arguments**

  * `f_0 = ones(m) ./ m` defines the prior, which is uniform by default.
  * `smoothing = Base.identity` is a function that optionally applies smoothing in between iterations. The operation is neither applied to the initial prior, nor to the final result. The function `inspect` is called before the smoothing is performed.
  * `K = 3` is the maximum number of iterations.
  * `epsilon = 0.0` is the minimum symmetric Chi Square distance between iterations. If the actual distance is below this threshold, convergence is assumed and the algorithm stops.
  * `stepsize = DEFAULT_STEPSIZE` is the step size taken in every iteration.
  * `fit_ratios = false` determines if ratios are fitted (i.e. `R` has to contain counts so that the ratio `f_est / f_train` is estimated) or if the probability density `f_est` is fitted directly.
  * `inspect = nothing` is a function `(f_k::Vector, k::Int, chi2s::Float64, alpha_k::Float64) -> Any` optionally called in every iteration.


In [12]:
?RUN

search: RUN run RunStepsize PRUN trunc truncate f_run round RoundUp f_p_run



```
RUN(binning; kwargs...)
```

The *Regularized Unfolding* method, using a `binning` to discretize the observable features.

**Keyword arguments**

  * `n_df = size(R, 2)` is the effective number of degrees of freedom. The default `n_df` results in no regularization (there is one degree of freedom for each dimension in the result).
  * `K = 100` is the maximum number of iterations.
  * `epsilon = 1e-6` is the minimum difference in the loss function between iterations. RUN stops when the absolute loss difference drops below `epsilon`.
  * `acceptance_correction = nothing`  is a tuple of functions (ac(d), inv*ac(d)) representing the acceptance correction ac and its inverse operation inv*ac for a data set d.
  * `ac_regularisation = true`  decides whether acceptance correction is taken into account for regularisation. Requires `acceptance_correction` != nothing.
  * `log_constant = 1/18394` is a selectable constant used in log regularisation to prevent the undefined case log(0).
  * `inspect = nothing` is a function `(f_k::Vector, k::Int, ldiff::Float64, tau::Float64) -> Any` optionally called in every iteration.
  * `fit_ratios = false` determines if ratios are fitted (i.e. `R` has to contain counts so that the ratio `f_est / f_train` is estimated) or if the probability density `f_est` is fitted directly.


In [13]:
?PRUN

search: PRUN f_p_run process_running alpha_adaptive_run permute! permutedims



```
PRUN(binning; kwargs...)
```

A version of the *Regularized Unfolding* method that is constrained to positive results. Like the original version, it uses a `binning` to discretize the observable features.

**Keyword arguments**

  * `tau = 0.0` determines the regularisation strength.
  * `K = 100` is the maximum number of iterations.
  * `epsilon = 1e-6` is the minimum difference in the loss function between iterations. RUN stops when the absolute loss difference drops below `epsilon`.
  * `f_0 = ones(size(R, 2))` Starting point for the interior-point Newton optimization.
  * `acceptance_correction = nothing`  is a tuple of functions (ac(d), inv*ac(d)) representing the acceptance correction ac and its inverse operation inv*ac for a data set d.
  * `ac_regularisation = true`  decides whether acceptance correction is taken into account for regularisation. Requires `acceptance_correction` != nothing.
  * `log_constant = 1/18394` is a selectable constant used in log regularisation to prevent the undefined case log(0).
  * `inspect = nothing` is a function `(f_k::Vector, k::Int, ldiff::Float64) -> Any` called in each iteration.
  * `fit_ratios = false` determines if ratios are fitted (i.e. `R` has to contain counts so that the ratio `f_est / f_train` is estimated) or if the probability density `f_est` is fitted directly.


In [14]:
?SVD

search: SVD f_svd isvalid CrossValidation



```
SVD(binning; kwargs...)
```

The *SVD-based* deconvolution method, using a `binning` to discretize the observable features.

**Keyword arguments**

  * `effective_rank = -1` is a regularization parameter which defines the effective rank of the solution. This rank must be <= dim(f). Any value smaller than one results turns off regularization.
  * `N = sum(g)` is the number of observations.
  * `B = DeconvUtil.cov_Poisson(g, N)` is the varianca-covariance matrix of the observed bins. The default value represents the assumption that each observed bin is Poisson-distributed with rate `g[i]*N`.
  * `epsilon_C = 1e-3` is a small constant to be added to each diagonal entry of the regularization matrix `C`. If no such constant would be added, inversion of `C` would not be possible.
  * `fit_ratios = false` determines if ratios are fitted (i.e. `R` has to contain counts so that the ratio `f_est / f_train` is estimated) or if the probability density `f_est` is fitted directly.
